In [102]:
import datetime

import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


import string
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from nltk.stem.snowball import SnowballStemmer



In [12]:
business = pd.read_json('yelp_dataset/business.json', lines=True)
#checkin = pd.read_json('yelp_dataset/checkin.json', lines=True)
#photo = pd.read_json('yelp_dataset/photo.json', lines=True)
#review = pd.read_json('yelp_dataset/review.json', lines=True)
#tip = pd.read_json('yelp_dataset/tip.json', lines=True)
##user = pd.read_json('yelp_dataset/user.json', lines=True)

MemoryError: 

## Helper Functino

In [61]:
def text_process(text):
    """
    Modified from
    http://adataanalyst.com/scikit-learn/countvectorizer-sklearn-example/
    Takes in a string of text, then performs the following:
    1. Remove all punctuation, and digits 
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    stemmer = SnowballStemmer("english")
   
    # Check characters to see if they are in punctuation
    clean = [char for char in text if (char not in string.punctuation) 
            and (not char.isdigit())] 
 
    clean = ''.join(clean)
    tokens = clean.split()
    tokens = [stemmer.stem(c) for c in tokens]
    # Join the characters again to form the string.

    tokens = ' '.join(tokens)
    
    # Now just remove any stopwords
    return tokens

In [ ]:
review.head()

### Only look at restaurants

In [3]:
#lv_restaurants
restaurants = business['categories'].str.contains('Restaurants', regex=False)
restaurants = restaurants.fillna(False)
restaurants = business[restaurants]
print(restaurants.shape)
restaurants.head()

(59371, 14)


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...",1Dfx3zM-rW4n-31KeC8sJg,"Restaurants, Breakfast & Brunch, Mexican, Taco...",Phoenix,"{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...",fweCYi8FmbJXHCqLnwuk8w,"Italian, Restaurants, Pizza, Chicken Wings",Mentor-on-the-Lake,"{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...",1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH
17,"1775 E Tropicana Ave, Ste 29","{'OutdoorSeating': 'False', 'BusinessAcceptsCr...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,None,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV


In [4]:
X_train, X_test, y_train, y_test = train_test_split(restaurants['categories'], \
                                                    restaurants['stars'], test_size=0.33, random_state=42)


In [5]:
X_train = pd.Series(X_train).str.get_dummies(sep=', ')
X_test = pd.Series(X_test).str.get_dummies(sep=', ')

In [7]:
### Vectorized value to get categories
#vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
#categories_count = vectorizer.fit_transform(restaurants['categories']).astype(np.int8)
#categories_list = vectorizer.get_feature_names()
#categories_df = pd.DataFrame(categories_count.toarray())
#categories_df.columns = categories_list
#categories_df.head()

In [9]:
#check for columns missing from test set
missing_train = list(set(list(X_train)) - set(list(X_test)))
for col in missing_train:
    X_test[col] = 0
    
missing_test = list(set(list(X_test)) - set(list(X_train)))
for col in missing_test:
    X_train[col] = 0

In [10]:
X_train.head()

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult Education,Adult Entertainment,Advertising,Afghan,African,...,Metal Fabricators,Rest Stops,Employment Agencies,Mortgage Brokers,Preschools,RV Parks,Game Meat,Gardeners,Carpet Installation,Auto Insurance
68223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42070,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_test.head()

,& Probates,Acai Bowls,Accessories,Acne Treatment,Active Life,Adult Entertainment,Afghan,African,Air Duct Cleaning,Aircraft Repairs,...,Art Supplies,Scottish,Historical Tours,Printing Services,Mass Media,Service Stations,Senegalese,Session Photography,Marinas,Pet Training
67682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print(X_train.shape)
print(X_test.shape)

(39778, 761)
(19593, 761)


In [16]:
y_train_cut = pd.cut(np.array(y_train), 2, labels=[1, 2])
y_test_cut = pd.cut(np.array(y_test), 2, labels=[1, 2])
np.array(y_train_cut)

array([2, 1, 2, ..., 1, 2, 2])

In [17]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, np.array(y_train_cut)) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [18]:
np.array(y_test_cut)

array([2, 2, 2, ..., 1, 1, 2])

In [19]:
neigh.score(X_test, np.array(y_test_cut))

0.5274332669831062

In [73]:
y_pred = neigh.predict(X_test)

In [74]:
accuracy_score(y_test_cut, y_pred)

0.5274332669831062

## Linear regression

#### Ridge

In [56]:
reg = linear_model.Ridge(alpha=.5)

In [57]:
reg.fit(X_train, y_train)


Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [58]:
y_pred = reg.predict(X_test)
y_pred = pd.Series(y_pred)

In [59]:
y_pred.shape

(19593,)

In [60]:
y_test.shape

(19593,)

In [61]:
mean_squared_error(y_test, y_pred)

1.568242542272463

In [62]:
r2_score(y_test, y_pred)  

-1.4283457838421336

#### Standard

In [66]:
regL = linear_model.LinearRegression()

In [67]:
regL.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [68]:
y_pred = regL.predict(X_test)
y_pred = pd.Series(y_pred)

In [69]:
y_pred.shape

(19593,)

In [70]:
y_test.shape

(19593,)

In [71]:
mean_squared_error(y_test, y_pred)

1.9275903467365447e+24

In [72]:
r2_score(y_test, y_pred)  

-2.984778033562134e+24

## Reviews

In [55]:
##The Whole dataset, but running into memory issues
#start = datetime.datetime.now()

#iter_review =  pd.read_json('yelp_dataset/review.json', lines=True, chunksize=1000):
#reviews_df = pd.concat([df[df['business_id'].isin(set(df['business_id']).intersection(set(restaurants['business_id'])))] for df in iter_review])

#end = datetime.datetime.now()
#elapse = end-start
#print(elapse.seconds)

KeyboardInterrupt: 

In [56]:
reviews_df = pd.DataFrame()
start = datetime.datetime.now()
for df in pd.read_json('yelp_dataset/review.json', lines=True,  chunksize=1000):
    reviews_df = df[df['business_id'].isin(set(df['business_id']).intersection(set(restaurants['business_id'])))]
end = datetime.datetime.now()
elapse = end-start
print(elapse.seconds)

366


In [57]:
reviews_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
6685000,pSMK_FtULKiU-iuh7SMKwg,0,2016-10-13 16:52:20,0,zLYNPgDU4_F50dQdx3TSoQ,5,- Made fresh right in front of you.\n- Build y...,0,XxakMGoMuleHaK8E8vjgPg
6685002,w55sCX7ZkLIJNhl_RhwjEg,0,2013-09-29 23:08:49,1,OAdpQth9cN-g0qOoKxEUMQ,3,I walked over from the Hard Rock and gave it a...,0,SSU1dl_alIw2lCphyE1WAw
6685003,HxfrVqxnsFI7MByNI7QINg,2,2013-08-01 02:38:57,0,jCoNDxz1NUElj9aI7re-Pw,4,I have been to a Joe's Crab Shack in New Jerse...,0,zX7hvqAKAD_sCbg7TaIIIw
6685006,DTz6gFeWmdvSEIFGjXpIgQ,0,2014-09-09 02:21:39,0,BfC8BsrKoWe_LP6tMtS_sQ,4,We came across Grinders by accident. I'm so g...,1,DQny4gJfywcCSMtbgY1oGg
6685007,wWvNr1sWlueHRqQrssc3iA,0,2018-02-23 14:06:17,0,RPnbi48dUdNy93vk-vhZjw,3,Let me start by saying that the service is alw...,0,kGNghQmR8SyGYAc6eOUJQw


In [94]:
X_train, X_test, y_train, y_test = train_test_split(text_sparse_df, \
                                                    reviews_df.stars, test_size=0.33, random_state=42)


In [69]:
## keep the size to 1000 features for size
reviews_df.text = reviews_df.text.apply(text_process)
vectorizer = TfidfVectorizer(analyzer='word',min_df=10, stop_words = stop_words.ENGLISH_STOP_WORDS, 
                             ngram_range=(1, 2))
text_tfidf = vectorizer.fit_transform(reviews_df.text)
text_features = vectorizer.get_feature_names()
text_sparse_df = pd.SparseDataFrame(text_tfidf)
text_sparse_df.columns = text_features
text_sparse_df.fillna(0.0, inplace=True)

C:\Users\vghli\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [95]:
X_train.head()

,abl,absolut,actual,ad,add,alreadi,alway,amaz,ambianc,ani,...,work,world,worst,worth,wouldnt,wrap,wrong,year,yelp,yes
335,0.06363,0.0,0.118623,0.0,0.0,0.062818,0.000000,0.0,0.0,0.052413,...,0.0,0.0,0.0,0.0,0.06363,0.071023,0.0,0.0,0.0,0.067403
312,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000
173,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000
238,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000
109,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.306418,0.0,0.0,0.177234,...,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000


In [97]:
reg = linear_model.Ridge(alpha=.5)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_pred = pd.Series(y_pred)

In [98]:
print("MSE:" + str(mean_squared_error(y_test, y_pred)))
print("R2:"+ str(r2_score(y_test, y_pred)))

MSE:1.3631144883360429
R2:0.4571507862015698


In [103]:
ridgereg = linear_model.Ridge()
alpha_range = [10, 1, 0.1, 0.01, 0.001]
ridge_params = {'alpha': alpha_range}
ridge_grid = GridSearchCV(ridgereg, ridge_params, cv=10, scoring='neg_mean_squared_error', n_jobs = 4, return_train_score=True)
ridge_grid.fit(X_train, np.ravel(y_train))
ridge_results = pd.DataFrame(ridge_grid.cv_results_)
ridge_results.sort_values(by='rank_test_score').head(5)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,0.222307,0.022639,-1.104938,-0.400692,1,{'alpha': 1},1,-0.998031,-0.405314,-0.932002,...,-0.859022,-0.414827,-1.069806,-0.398082,-1.175630,-0.394780,0.018462,0.008285,0.138553,0.006576
2,0.243656,0.019740,-1.137311,-0.046049,0.1,{'alpha': 0.1},2,-1.079784,-0.046869,-1.225148,...,-0.760269,-0.049472,-1.263869,-0.044915,-1.197591,-0.047771,0.020108,0.002479,0.181811,0.003271
3,0.237368,0.019644,-1.439217,-0.001436,0.01,{'alpha': 0.01},3,-1.426345,-0.001457,-1.598822,...,-1.125729,-0.001570,-1.657883,-0.001375,-1.428999,-0.001620,0.031362,0.003395,0.240843,0.000178
4,0.220412,0.018350,-1.534004,-0.000018,0.001,{'alpha': 0.001},4,-1.529518,-0.000018,-1.701607,...,-1.245906,-0.000020,-1.765783,-0.000017,-1.522985,-0.000020,0.018444,0.000914,0.246638,0.000002
0,0.225793,0.016853,-1.716923,-1.447801,10,{'alpha': 10},5,-1.604702,-1.461900,-1.138059,...,-1.634531,-1.458094,-1.631740,-1.456010,-1.721061,-1.443278,0.077007,0.002017,0.239471,0.018477


In [99]:
regL = linear_model.LinearRegression()
regL.fit(X_train, y_train)
y_pred_regL = regL.predict(X_test)
y_pred_regL = pd.Series(y_pred)

In [100]:
print("MSE:" + str(mean_squared_error(y_test, y_pred_regL)))
print("R2:"+ str(r2_score(y_test, y_pred_regL)))

MSE:1.3631144883360429
R2:0.4571507862015698
